<center><font size = "10"> Week 6 - Single Cell Electrophysiology<center>
<center><font size = "8">Tutorial 02: Analysis<center>

<font size = "3"><font color = "blue"> In this tutorial you will analyse real single cell voltage recordings

In [ ]:
# Load usefull packages
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import csv
import pandas as pd

# 1. Open files with traces

### Traces description

<font size='3'>Here we have some traces (exp_FirePatter_ch(6/7)_(52/53).dat). Ch6 contains the voltage response and Ch7 the current stimulation. The numbers 52 and 53 represent different "sweeps", so different repetitions of the same stimulation over the same cell.

In [ ]:
# Open and plot files

def get_data(fn):
    """Read  data file and formatfor plotting"""
    d = np.fromfile(fn)    
    return d[::2], d[1::2]

file_v1 = "exp_FirePattern_ch6_52.dat"
file_v2 = "exp_FirePattern_ch6_53.dat"

FILE_v = [file_v1, file_v2] 

plt.figure(figsize=(15,3))
for fv in FILE_v:
    t, v = get_data(fv)
    plt.plot(t,v)

# 2. Extracting peaks, peak times and amplitude for one sweep

In [ ]:
from scipy.signal import find_peaks, peak_prominences

t, v = get_data(file_v1)

# Amplitudes and peak locations
peaks_time, _ = find_peaks(v, prominence=1, height=-20)
amplitudes = peak_prominences(v, peaks_time, wlen=40)[0]

voltage_peaks = []
for x in peaks_time:
    volt_p = v[x]
    voltage_peaks.append(volt_p)
    
#Print results   
print("Voltage peaks: %s" %voltage_peaks)
print('\n')
print ("Amplitudes of action potentials: %s " % amplitudes)
print('\n')
print("Times of peak occurences: %s" % peaks_time)

# Plot
plt.figure(figsize=(15,3))
plt.plot(v,'k--')
plt.plot(peaks_time, v[peaks_time], 'rx')
plt.ylabel('voltage (mV)')

# 3. Using a specific library: eFEL

<font size = "3">Find the project [here](https://github.com/BlueBrain/eFEL) and the documentation [here](https://efel.readthedocs.io/en/latest/) (also in [pdf](https://readthedocs.org/projects/efel/downloads/pdf/latest/))

<font size='3'>We will use the getFeatureValues() function, which accepts as input a dictionary that represents the trace, and the request eFeature names. The trace dictionary consists of the time and voltage trace, and two floats stim_start and stim_end that specify when the stimulus starts and stops.

In [ ]:
file_c1 = "exp_FirePattern_ch7_52.dat"

t, v = get_data(file_v1)
t, i = get_data(file_c1)

# Look for start and end time
peaks_time, _ = find_peaks(i, prominence=1, height=-20)

print(np.min(peaks_time))
print(np.max(peaks_time))

In [ ]:
import efel
from json2html import *
import IPython

stim_start = 378.9 # in ms
stim_end = 3681.0
trace = {'T': t, 'V': v, 'stim_start': [stim_start], 'stim_end': [stim_end]}

In [ ]:
feature_values = efel.getFeatureValues([trace], ['AP_amplitude', 'peak_time', 'peak_voltage'])[0]
feature_values = {feature_name: list(values) for feature_name, values in feature_values.items()}
IPython.display.HTML(json2html.convert(json=feature_values))